In [28]:
import torchaudio as ta
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
# import pytorch_lightning as pl
import numpy as np
import os

import librosa

import IPython.display as ipd

import numpy as np

import tensorflow as tf

import pandas as pd

import glob

from tqdm.auto import tqdm

from Noise_Reduction_Datagen_numpy_ver import Signal_Synthesis_DataGen
from Noise_Data_Loader import Noise_Data_loader

import warnings
warnings.filterwarnings("ignore")

import gc

# from numba import jit

In [40]:
noise_dir = "./dataset/UrbanSound8K/all_files/"
signal_dir = "./dataset/cv-corpus-5.1-2020-06-22/en/clips/"
signal_nums_save = "./dataset_loader_files/signal_paths_nums_save.npy"
num_noise_samples=200
num_signal_samples = 10000
noise_save_path = "./dataset_loader_files/noise_paths_save.npy"
n_fft=1024
win_length=256
hop_len=128
default_sr = 16000
sec = 6
augment=True

signal_synthesis_dataset = Signal_Synthesis_DataGen(noise_dir, signal_dir, \
                signal_nums_save=signal_nums_save, num_noise_samples=num_noise_samples, \
                num_signal_samples=num_signal_samples, noise_path_save=noise_save_path, \
                 n_fft=n_fft, win_length=win_length, hop_len=hop_len, f_min=0, f_max=8000, \
                 perform_stft=True, normalize=True, default_sr=16000, sec=6, augment=False)

Loading noise from saved file
Loading nums from npy file
10000


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 8
shuffle = True
num_workers = 14
# data_loader = DataLoader(signal_synthesis_dataset, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=num_workers)
data_loader = DataLoader(signal_synthesis_dataset, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=num_workers)
# data_loader.to(device)

In [42]:
signal_synthesis_dataset.__len__()

2000000

In [43]:
%%time
# data_loader_iter = iter(data_loader)
for index, i in enumerate(data_loader):
#     i = next(data_loader)
    print(i[0].shape, index)
    if index < 2000:
        continue
    break

torch.Size([8, 513, 751]) 0
torch.Size([8, 513, 751]) 1
torch.Size([8, 513, 751]) 2
torch.Size([8, 513, 751]) 3


KeyboardInterrupt: 

In [44]:
np.floor(((default_sr*sec) - (win_length - 1) - 1)/ hop_len + 3)

751.0

In [4]:
# Model building
# class BiDirectLSTM(nn.Module):
#     def __init__(self, rnn_dim, hidden_size, dropout, batch_first=True):
#         super(BiLSTM, self).__init__()
        
#         self.BiLSTM = nn.LSTM(
#                 input_size=rnn_dim,
#                 hidden_size=hidden_size,
#                 num_layers=2, batch_first=batch_first, bidirectional=True)
#         self.layer_norm = nn.LayerNorm(rnn_dim)
#         self.dropout(dropout)
        
#     def forward(self, x):
#         x = self.layer_norm(x)
#         x = F.gelu(x)
#         x, _ = self.BiLSTM(x)
#         x = self.dropout(x)
#         return x
        


class NoiseReducer(nn.Module):
    def __init__(self, default_sr, n_fft, win_length, hop_len, f_min, f_max, dropout=0.5, batch_first=True, stride=2, normalized=True):
        super(NoiseReducer, self).__init__()
        
        self.default_sr = default_sr
        self.n_fft = n_fft
        self.win_length = win_length
        self.hop_len = hop_len
        self.f_min = f_min
        self.f_max = f_max
        self.normalized = normalized
        
        # LSTM UNITS
        self.rnn_dims = np.floor(((default_sr*sec) - (win_length - 1) - 1)/ hop_len + 3)
        self.num_layers = 4
        self.batch_first = True
        self.dropout = 0.2
        self.bidir = True
        
        # Dense Layer Units
        self.inp_dims = n_fft // 2 + 1

        
        
        self.bilstm1 = nn.LSTM(input_size=self.rnn_dims, hidden__state=self.rnn_dims, num_layers=4, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        self.bilstm2 = nn.LSTM(input_size=self.rnn_dims, hidden__state=self.rnn_dims, num_layers=4, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        self.dense1 = nn.Linear()
        
    def forward(self, inp_tensor):
        inp_tensor = inp_tensor.permute(0, 2, 1)
        x, h_n, c_n = self.bilstm1(inp_tensor)
        x, h_n, c_n = self.bilstm2(x, h_n, c_n)
        
        return x
        
        

In [5]:
lin_layer = nn.Linear(128, 64)
inp = torch.randn((4, 5, 128))
out = lin_layer(inp)

In [6]:
out.shape

torch.Size([4, 5, 64])

In [7]:
96000/200

480.0

In [11]:
i

NameError: name 'i' is not defined

In [ ]:
i[0].min()

In [ ]:
len(data_loader)

In [ ]:
gc.collect()

In [ ]:
batch = data_loader.__iter__()

In [ ]:
batch[0].shape